In [1]:
from pandas.io import sql
from sqlalchemy import create_engine, inspect

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
db = create_engine('sqlite:////share/morePicturesOfCats.db')

In [8]:
df = pd.read_csv("HousingSLO-Master.csv")

In [9]:
df.head()

,MLSNumber,Street,City,BR,Bath,Footage,Price,PriceSqFt,County,Zone,Source,Date,URL
0,1072327,629 Northbrook,Lompoc,3,3,1652,370000,223.970944,SB,0,0,October 23,http://www.slocountyhomes.idxbroker.com/idx/d...
1,1072320,1249 Westbrook,Lompoc,3,2,1791,389000,217.197097,SB,0,0,October 23,http://www.slocountyhomes.idxbroker.com/idx/d...
2,1072319,247 Brisa Del Mar,Lompoc,3,2,2360,518000,219.491525,SB,0,0,October 23,http://www.slocountyhomes.idxbroker.com/idx/d...
3,1072317,2760 Cedar,Morro Bay,2,2,976,550000,563.524590,SLO,1,0,October 23,http://www.slocountyhomes.idxbroker.com/idx/d...
4,1072288,1007 Jacqueline,Nipomo,3,3,2325,699900,301.032258,SLO,0,0,October 23,http://www.slocountyhomes.idxbroker.com/idx/d...


In [20]:
df.to_sql('housing', db)

ValueError: Table 'housing' already exists.

In [23]:
df = df.iloc[:, 2:]

In [22]:
df = pd.read_sql_query("select * from housing", db)

In [ ]:
# generate more features, like zip code

In [ ]:
# have a list of classifiers we will use
# write model functions that will return score of model and the error, and computation time
# Logistic Regression - multiclass
# kNN
# AdaBoostClassifier
# Random Forest
# DecisionTreeClassifier
# SVM
# ensemble
    # Voting

In [ ]:
def compareModels(X, Y):
    """
    Y is the response variable
    X is vector of all the features
    Returns list(Model Name, score, error, computation time)
    """
    # calls predefined set of classifiers
    pass

In [ ]:
def expandColumns(dataFrame, colNames):
    """
    dataFrame: the data resides
    colName: list of columns to expand
    """
    pass

In [ ]:
def plotResult(results):
    """
    Take in a list of results 
    list(Model Name, score, error, computation time)
    display a visual representation of the result
    """
    pass